In [1]:
import sys
sys.path.append("/Colorization/src")  # Append path to src
import time
import numpy as np

In [2]:
from networks.dataset import load_dataset

In [3]:
root = '/Colorization/data/train2017'
annFile = '/Colorization/data/annotations/instances_train2017.json'
batch_size = 64
train_dataset = load_dataset(root, annFile, batch_size)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-frt5fsbs because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


loading annotations into memory...
Done (t=10.09s)
creating index...
index created!


In [4]:
import torch
pytorch_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
from networks.eccv16 import eccv16

model = eccv16(pretrained=False).to(pytorch_device)

criterion = torch.nn.MSELoss(reduction='sum')
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=3*1e-5,betas=(0.9,0.99),weight_decay=1e-3)

In [6]:
tic = time.time()
for i, (img_l, y) in enumerate(train_dataset):
    img_l = img_l.to(pytorch_device)
    y = y.to(pytorch_device)
    y_pred = model(img_l)
    print(np.linalg.norm(y_pred.detach().cpu().numpy() - y.cpu().numpy()))

    # Compute and print loss
    loss = criterion(y_pred, y)
    #loss = criterion(model.normalize_ab(y_pred), model.normalize_ab(y))
    
    toc = time.time()
    print(i, loss.item(), f'Time: {toc-tic} sec')
    tic = time.time()

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


48900.44
0 2391416832.0 Time: 2.087425708770752 sec
45685.168
1 2087234432.0 Time: 8.31327772140503 sec
50968.28
2 2597967104.0 Time: 6.674348592758179 sec
42881.805
3 1838942208.0 Time: 6.67439341545105 sec
45425.117
4 2063548032.0 Time: 6.694308042526245 sec
40564.984
5 1645593216.0 Time: 6.673346757888794 sec


KeyboardInterrupt: 